In [231]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from abc import ABC, abstractmethod
from scipy import constants
from tqdm import tqdm
from IPython.display import clear_output
import time

In [27]:
class InteractionEnergy(ABC):
    """
    Interface for computing interaction energies.
    """
    SORBENT = 'sorbent'
    
    @abstractmethod
    def get_energy_between(self, particle1, particle2, distance):
        """
        Get interaction energy (eV) between particle1 and particle2.

        Parameters:
            particle1, particle2: name of particles. "sorbent" for sorbent.
            distance: distance between particle1 and particle2

        Returns:
            interaction energy (eV).
        """
        pass

    @abstractmethod
    def get_energies_between(self, pairs):
        """
        Get interaction energies (eV) for each pair of particles in pairs.
        
        Parameters:
            pairs:
                tuples of particles for which to compute interaction energies.
                (name1, name2, distance)
        
        Returns:
            interaction energies (eV).
        """
        pass

In [71]:
class SimpleInteractionEnergy(InteractionEnergy):
    """
    Defines an energy scheme where interaction has a fixed energy.
    """
    def __init__(self, particles, energy_matrix):
        """
        SimpleInteractionEnergy(particles, energy_matrix)

        Parameters:
            particles: array of particle names
            energies: 
                2D array of interaction energies. energies[i][j] is the energy of particle i interacting with particle j.
                shape = (n+1)*(n+1) where n is the number of particles.
                ith column corresponds to particles[i] if i<n; corresponds to the sorbent if i==n.
                Same for rows.
                entries: energy (epsilon, eV).
        """
        energy_matrix = np.array(energy_matrix)
        if (energy_matrix.shape != (len(particles)+1, len(particles)+1)):
            raise ValueError(f"energy_matrix must have shape {(len(particles), len(particles)+1)}, got {energy_matrix.shape}")
        self.particles = {name: i for i, name in enumerate(particles)}
        self.particles[InteractionEnergy.SORBENT] = len(particles)
        self.energy_matrix = energy_matrix

    def get_energy_between(self, particle1, particle2, distance=0):
        
        idx1 = self.particles.get(particle1, -1)
        idx2 = self.particles.get(particle2, -1)
        if (idx1<0 and idx2<0):
            raise ValueError(f"{particle1} and {particle2} do not exist in the system.")
        if (idx1<0):
            raise ValueError(f"{particle1} does not exist in the system.")
        if (idx2<0):
            raise ValueError(f"{particle2} does not exist in the system.")
        
        return self.energy_matrix[idx1][idx2]

    def get_energies_between(self, pairs):
        
        return [self.get_energy_between(pair) for pair in pairs]

In [72]:
SimpleInteractionEnergy({'A', 'B'}, [
    [1, 2, 0],
    [2, 3, 5],
    [0, 5, 0]
]).get_energy_between(InteractionEnergy.SORBENT, 'B')

np.int64(5)

In [141]:
class Lattice(ABC):
    """
    Interface for representing a lattic surface
    """    
    EMPTY = 'empty'
    OCCUPIED = 'occupied'
    ALL = 'all'

    @abstractmethod
    def clear(self):
        """
        Clears the lattice in place.
        """
        pass
    
    @abstractmethod
    def get_n_sites(self, site_type):
        """
        Gets number of sites with the given type.

        Parameters:
            site_type: particle name (str or list-like)|Lattice.EMPTY|Lattice.OCCUPIED|Lattice.ALL
                Specifies the type of site using a string

        Returns:
            number of sites
        """
        pass
    
    @abstractmethod
    def draw_site(self, draw_type, rng):
        """
        Randomly draws an empty site.

        Parameters:
            draw_type: particle name (str)|Lattice.EMPTY|Lattice.OCCUPIED|Lattice.ALL
                Specifies the type of site using a string
            rng: a random number generator

        Returns:
            tuple,
            first entry: particle at site,
            sedond entry: internal representation of a site (you can understand as Site objects)
        """
        pass
    
    @abstractmethod
    def add_particle(self, site, particle):
        """
        Adds particle to site.

        Parameters:
            site: internal representation of the target site
            particle: name of particle to be added

        Returns:
            self
        """
        pass

    @abstractmethod
    def remove_particle(self, site, particle):
        """
        Adds particle to site.

        Parameters:
            site: internal representation of the target site
            particle: name of particle to remove

        Returns:
            self
        """
        pass

    @abstractmethod
    def get_energy(self, InteractionEnergy):
        """
        Computes surface energy.

        Parameters:
            InteractionEnergy: an InteractionEnergy object

        Returns:
            surface energy (eV)
        """
        pass

In [219]:
class SquareLattice(Lattice):
    """
    Defines a simple square lattice.
    """
    def __init__(self, n, cell_length=0):
        """
        SquareLattice(n)

        Parameters:
            n: number of cells per side
            cell_length (angstrom): length of each cell, 0 if not considering length
        """
        self.n = n
        self.lattice = np.array([[None]*n]*n, dtype=object)
        self.particles = set()
        self.cell_length = cell_length

    def clear(self):
        self.lattice.fill(None)
        self.particles.clear()

    def get_sites(self, site_type=Lattice.EMPTY):
        if site_type==Lattice.ALL:
            target_sites = [(0,0) for _ in range(self.n)**2]
            count = 0
            for i in range(self.n):
                for j in range(self.n):
                    target_sites[count] = (i, j)
                    count+=1
            return target_sites
        elif site_type==Lattice.EMPTY:
            target=[None]
        elif site_type==Lattice.OCCUPIED:
            target=self.particles
        else:
            target = np.array([site_type]).flatten()
        target_sites = []
        for i in range(self.n):
            for j in range(self.n):
                if self.lattice[i][j] in target:
                    target_sites.append((i, j))
        return target_sites

    def get_n_sites(self, site_type=Lattice.EMPTY):
        if site_type==Lattice.ALL:
            return self.n**2
        return len(self.get_sites(site_type=site_type))

    def draw_site(self, draw_type=Lattice.EMPTY, rng=None):
        target_sites = self.get_sites(site_type=draw_type)
        length = len(target_sites)
        if length==0:
            return (None, None)
        if rng == None:
            draw_idx = int(np.floor(np.random.random()*length))
        else:
            draw_idx = int(np.floor(rng.random()*length))
        site = target_sites[draw_idx]
        return (self.lattice[site[0]][site[1]], site)
        
    def add_particle(self, site, particle):
        if self.lattice[site[0]][site[1]] != None:
            raise ValueError(f"{particle} cannot be added to {site} as it is still occupied by {self.lattice[site]}.")
        self.lattice[site[0]][site[1]] = particle
        self.particles.add(particle)
        return self

    def remove_particle(self, site, particle=None):
        if self.lattice[site[0]][site[1]] == None:
            raise ValueError(f"Cannot remove from an empty site: {site}")
        if not particle==None and not particle==self.lattice[site[0]][site[1]]:
            raise ValueError(f"Cannot remove {particle} from site {site}, which has {self.lattice[site[0]][site[1]]}")
        self.lattice[site[0]][site[1]] = None
        return self

    def get_energy(self, interactionEnergy):
        """
        Computes surface energy considering only neighboring cells using 4-connectivity.
        Considers minimum image convention.

        Parameters:
            interactionEnergy: an InteractionEnergy object

        Returns:
            surface energy (eV)
        """
        energy = 0
        # we calculate total energy in one pass
        # by considering the right and bottom neighbors for each cell,
        # which incorporates all pairs of interactions and does not double count
        for i in range(self.n):
            for j in range(self.n):
                cur = self.lattice[i][j]
                if not cur==None:
                    # sorption energy
                    energy += interactionEnergy.get_energy_between(
                        cur, InteractionEnergy.SORBENT, self.cell_length
                    )
                    # sorbate interaction energy
                    nei1 = self.lattice[i][(j+1)%self.n]
                    if not nei1==None:
                        energy += interactionEnergy.get_energy_between(cur, nei1, self.cell_length)
                    nei2 = self.lattice[(i+1)%self.n][j]
                    if not nei2==None:
                        energy += interactionEnergy.get_energy_between(cur, nei2, self.cell_length)
        return energy

In [252]:
class GrandCanonicalEnsemble:
    """
    Defines a grand canonical ensemble for Monte Carlo simulation
    """    
    def __init__(self, particles, mus, temp, lattice, interactionEnergy, seed):
        """
        GrandCanonicalEnsemble(particles, mus, temp, lattice, interactionEnergy, seed)

        Parameters:
            particels: array of particle names
            mus (eV/mol): array of chemical potentials
            temp (K): simulation temperature
            lattice: a Lattice object that defines the simulation lattice
            interactionEnergy: an InteractionEnergy object that defines interaction energies
            seed: random seed
        """
        self.lattice = lattice
        self.interactionEnergy = interactionEnergy
        self.temp = temp
        self.beta = 1/(constants.k/constants.eV*temp) # eV^{-1}
        self.particles = particles
        self.mus = mus
        self.mu_dict = {particle: mu for particle, mu in zip(particles, mus)}
        self.rng = np.random.default_rng(seed)

    def reseed(seed):
        """
        Reseeds the current instance
        """
        self.rng = np.random.default_rng(seed)
    
    def attempt_add(self, particle, site):
        """
        Attempt adding a particle at site

        Parameters:
            particle: name of particle to add
            site: internal representation of a site
            
        Side effects: Updates self.lattice in place with added particle

        Returns:
            True if added successfully, False otherwise
        """
        Na = self.lattice.get_n_sites(Lattice.EMPTY)
        Ns = self.lattice.get_n_sites(particle)
        mu = self.mu_dict[particle]

        self.lattice.add_particle(site, particle)
        updated_surface_energy = self.lattice.get_energy(self.interactionEnergy)
        delta_e = updated_surface_energy - self.current_energy
        acc = min(1, (Na)/(Ns+1)*np.exp(-self.beta*(delta_e-mu)))
        prob = self.rng.random()
        if (prob<acc):
            self.current_energy = updated_surface_energy
            return True
        else:
            self.lattice.remove_particle(site, particle)
            return False

    def attempt_remove(self, particle, site):
        """
        Attempt removing a particle at site

        Parameters:
            particle: name of particle to add
            site: internal representation of a site

        Side effects: Updates self.lattice in place with removed particle

        Returns:
            True if removed successfully, False otherwise
        """
        Na = self.lattice.get_n_sites(Lattice.EMPTY)
        Ns = self.lattice.get_n_sites(particle)
        mu = self.mu_dict[particle]

        self.lattice.remove_particle(site, particle)
        updated_surface_energy = self.lattice.get_energy(self.interactionEnergy)
        delta_e = updated_surface_energy - self.current_energy
        acc = min(1, Ns/(Na+1)*np.exp(-self.beta*(delta_e+mu)))
        prob = self.rng.random()
        if (prob<acc):
            self.current_energy = updated_surface_energy
            return True
        else:
            self.lattice.add_particle(site, particle)
            return False

    def run(self, n_steps=200, move_probs=[0.5, 0.5], particle_probs=[0.5, 0.5], verbose=False):
        """
        Runs the Monte Carlo simulation and store the results.
        Updates self.coverages to the coverage of each particle at each step.

        Parameters:
            n_steps: steps to run
            move_probs: array of probabilities. [P(add), P(remove)]
            particle_probs: array of probabilities for operation on each particles.
            verbose: sets verbose output

        Returns: self
        """
        self.lattice.clear()
        self.current_energy = self.lattice.get_energy(self.interactionEnergy)
        coverages = np.zeros((n_steps, len(self.particles)))
        move_probs = np.cumsum(move_probs)
        particle_probs = np.cumsum(particle_probs)

        if verbose:
            itr = range(n_steps)
        else:
            itr = tqdm(range(n_steps))
        for step in itr:
            prob = self.rng.random()
            # add
            if prob<move_probs[0]:
                particle_idx = int(self.rng.random()*len(self.particles))
                _, site = self.lattice.draw_site(draw_type=Lattice.EMPTY, rng=self.rng)
                if not site==None:
                    self.attempt_add(self.particles[particle_idx], site)
            # remove
            else:
                particle, site = self.lattice.draw_site(draw_type=Lattice.OCCUPIED, rng=self.rng)
                if not site==None:
                    self.attempt_remove(particle, site)
            for j in range(coverages.shape[1]):
                coverages[step][j] = self.lattice.get_n_sites(self.particles[j])/self.lattice.get_n_sites(Lattice.ALL)
            if verbose:
                clear_output(wait=True)
                print(f"Step {step}, coverages: {coverages[step]}")
                print(self.lattice.lattice, end='\r', flush=True)
                time.sleep(0.05)
        self.coverages = coverages
        return self

In [269]:
particles = ['A', 'B']
mus = [-0.1, -0.15]
temp = 298
n = 6
move_probs = [0.5, 0.5]
particle_probs = [0.5, 0.5]

In [270]:
#          A  B  sorbent
# A
# B
# sorbent
simple_energy_mat = np.array([
    [0,    0,    -0.1],
    [0,    0,    -0.1],
    [-0.1, -0.1, 0   ]
])
simpleInteraction = SimpleInteractionEnergy(particles, simple_energy_mat)

In [271]:
squareLattice = SquareLattice(n)
ensemble = GrandCanonicalEnsemble(
    particles,
    mus,
    temp,
    squareLattice,
    simpleInteraction,
    114514
)

In [272]:
ensemble.run(n_steps=200, move_probs=move_probs, particle_probs=particle_probs, verbose=True)

Step 199, coverages: [0.27777778 0.13888889]
[[None 'A' None 'A' None 'A']
 [None None 'A' None 'A' 'A']
 ['B' None None None 'A' None]
 ['B' None None 'B' None 'A']
 [None None None None 'B' None]
 ['A' 'A' None None None 'B']]